# 6. 转换任务（Transforming）

大型语言模型非常擅长将输入转换为不同的格式。

例如输入一种语言的文本，将其转换或翻译为另一种语言，或者帮助进行拼写和语法的检查和修改。因此，将一段不完全符合语法的文本作为输入，可以让它帮助你x纠正拼写和语法。或者用来转换文本格式，例如输入 HTML ，让它输出 JSON 格式的文本。

我以前编写应用程序的时候，要非常辛苦编写一堆正则表达式。现在通过大语言模型和一些提示，就可以更简单地实现。

是的，我现在基本上使用 ChatGPT 来校对我写的任何东西，所以我很高兴能向你展示 Notebook 中的更多例子。

## 6.1 文本翻译

ChatGPT使用多种语言的源代码进行训练。这使模型能够进行翻译。以下是一些如何使用此功能的示例。

首先，我们导入 OpenAI，使用我们在本视频中一直使用的 get_completion 辅助函数。

In [ ]:
import openai
import os
from openai import OpenAI

# 1. 根据环境变量获取 openai key
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY') 

client = OpenAI()

# 2. 定义 get_completion 方法
def get_completion(prompt, instructions=None, model="gpt-3.5-turbo"):
    response = client.responses.create(
        model=model,
        instructions=instructions,
        input=prompt,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.output_text

我们要做的第一件事是翻译任务。大型语言模型是在许多来源的大量文本上训练出来的，其中很多内容来自互联网，这当然会有许多不同的语言。因此， 这使模型具有翻译能力。模型以不同程度的熟练掌握数百种语言。我们将通过一些例子来介绍如何使用这种能力。

让我们从简单的问题开始。在第一个例子中，提示是将以下英文文本翻译成西班牙语： “Hi, I would like to order a blender”。

In [ ]:
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like to order a blender```
"""

response = get_completion(prompt)

print(response) 

模型的预期回答是“Hola，me gustaría ordenar una licuadora”。

很遗憾，我没学过西班牙语，你肯定能看出来。

好，让我们尝试另一个例子。在这个例子中，提示是，告诉我这是什么语言。然后这是一句法语 “Combien coûte la lampe d'air”。

In [ ]:
prompt = f"""
Tell me which language this is: 
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response) 

我们来运行一下。

This is French.

模型已经识别出这是法语。

模型也可以同时进行多种翻译。在这个例子中，提示要求，将以下文本翻译成法语和西班牙语，再加一个“海盗英语”。这段文本是，“我想订购一个篮球”。

In [ ]:
prompt = f"""
Translate the following text to French and Spanish
and English pirate: \
```I want to order a basketball```
"""
response = get_completion(prompt)
print(response) 

模型的输出，这里是法语，西班牙语，还有海盗英语。

French: ```Je veux commander un ballon de basket```

Spanish: ```Quiero ordenar un balón de baloncesto```

English pirate: ```I be wantin' to order a basketball```

在一些语言中，翻译可能会因说话者与听众的关系而变化。你也可以向语言模型解释这一点，这样它就能进行相应的翻译。

在这个例子中，我们提示要求，将以下文本翻译成西班牙语，分别用正式的和非正式的用法表达，“你想订购一个枕头吗？”。

In [ ]:
prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response) 

请注意，为了进行区别，我们在这里使用了不同于重音符的分隔符，而不是双引号。使用什么分隔符并不重要，只要能实现清晰的分隔就可以。

Formal: ¿Le gustaría ordenar una almohada?

Informal: ¿Te gustaría ordenar una almohada?

模型的输出，在这里有正式和非正式用法的区别。正式用法是指当你和比你资深的人交谈或者在专业环境下使用的语气，而非正式用法是指你和朋友说话时所使用的语气。我其实不会说西班牙语，但是我爸爸会，他说这是正确的。

## 6.2 通用翻译器

下一个例子，假设我们负责一家跨国电商公司，用户发来的信息将会是各种不同的语言，因此他们会用各种不同的语言，告诉我们关于 IT 的问题。因此，我们需要一个通用的翻译器。

首先，我们将粘贴一个各种不同语言的用户信息的列表，然后我们将循环遍历每一条用户消息。

In [ ]:
user_messages = [
    "La performance du système est plus lente que d'habitude.", # System performance is slower than normal 
    "Mi monitor tiene píxeles que no se iluminan.", # My monitor has pixels that are not lighting
    "Il mio mouse non funziona", # My mouse is not working
    "Mój klawisz Ctrl jest zepsuty", # My keyboard has a broken control key
    "我的屏幕在闪烁" # My screen is flashing
]  

对于用户消息中的问题，我将复制这个稍长一点的代码块。我们首先让模型告诉我们，这个问题用的是什么语言，然后打印出原始消息使用的语言和问题的内容，然后我们要求模型将其翻译成英语和韩语。

In [ ]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following text to English \
    and Korean: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n") 

让我们运行一下。

Original message (This text is in French.): La performance du système est plus lente que d'habitude.

English: "The system performance is slower than usual."

Korean: "시스템 성능이 평소보다 느립니다." 

Original message (This sentence is in Spanish.): Mi monitor tiene píxeles que no se iluminan.

English: "My monitor has pixels that do not light up."

Korean: "내 모니터에는 빛나지 않는 픽셀이 있습니다." 

Original message (This phrase is in Italian. It translates to "My mouse is not working" in English.): Il mio mouse non funziona

English: My mouse is not working

Korean: 내 마우스가 작동하지 않아요 

Original message (This text is in Polish.): Mój klawisz Ctrl jest zepsuty

English: My Ctrl key is broken

Korean: 제 Ctrl 키가 고장 났어요 

Original message (This text is in Chinese.): 我的屏幕在闪烁

English: My screen is flickering

Korean: 내 화면이 깜박거립니다

模型的输出是，这条原始消息是法语，还有各种语言的消息，然后模型将它们翻译成英语和韩语。你可以在这里看到，模型的输出是 “This is French”， 这是因为此在提示中要求的响应格式是“This is French”。如果你希望只用一个单词或不用句子来回答，你可以试着编辑这个提示。或者你也可以要求它以 JSON 格式或类似的方式，这将会鼓励它不要使用整个句子来回答。

令人惊叹的是，你刚刚构建了一款通用翻译器。你可以随时暂停视频，在这里添加任何你想尝试语言，也许是你自己说的语言，看看模型的表现如何。

## 6.3语气和风格变换

ChatGPT可以产生不同的风格（语气）。

接下来我们要深入探讨的是风格转换。

写作可以根据预期的受众不同而变化，我给同事或教授写邮件的方式，显然会与我给弟弟发短信的方式大不相同。ChatGPT 也可以帮助产生不同的语气。

让我们看一些例子。在第一个例子中，提示是，将以下俚语翻译成商业信函：“老兄，这是乔，看看这盏落地灯的规格。”

In [ ]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response) 

我们来执行一下。

Dear Sir/Madam,

I am writing to bring to your attention the specifications of the standing lamp. 

Sincerely,

Joe

正如你所看到的，我们得到了一封更正式的商业信函，提出关于落地灯规格的建议。

## 6.4 文本格式转换

接下来我们要做的是在不同的格式之间进行转换。

ChatGPT 非常擅长在不同的格式之间进行转换，比如从 JSON 到 HTML，XML，markdown，等等。 在提示中，我们将描述输入和输出格式。这里有一个例子。因此，我们一个 JSON 格式，包含一个餐厅员工的名单，包括他们的名字和电子邮件。

在提示中，我们要求模型将其从 JSON 转换为 HTML，提示是：将以下的 Python 字典从 JSON 转换为具有列头和标题行的 HTML 表格。然后我们将从模型中获得响应并将其打印出来。

In [ ]:
data_json = { 
    "resturant employees" :[ 
        {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
        {"name":"Bob", "email":"bob32@gmail.com"},
        {"name":"Jai", "email":"jai87@gmail.com"}
    ]
}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response) 

模型的输出如下。

```html
 <table>
 <caption>Restaurant Employees</caption>
 <thead>
 <tr>
 <th>Name</th>
 <th>Email</th>
 </tr>
 </thead>
 <tbody>
 <tr>
 <td>Shyam</td>
 <td>shyamjaiswal@gmail.com</td>
 </tr>
 <tr>
 <td>Bob</td>
 <td>bob32@gmail.com</td>
 </tr>
 <tr>
 <td>Jai</td>
 <td>jai87@gmail.com</td>
 </tr>
 </tbody>
</table>
```

我们得到了HTML格式，显示所有员工的名字和电子邮件。让我们看看是否可以实际查看这个 HTML。我们将使用 Python 库中的显示函数，来显示 HTML 响应。

In [ ]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response)) 

预期可以看到下面格式的 HTML 表格。

![html](./img/6-1.jpg)

## 6.5 拼写检查/语法检查

我们的下一个转换任务是拼写检查和语法检查。

这是 ChatGPT 的一个非常流行的用途。我强烈推荐这样做。我一直都这样做。当你在非母语语言中工作时，特别有用。

这里有一些常见的语法和拼写问题的例子，这个例子展示语言模型如何帮助解决这些问题。

我将粘贴一个有一些语法或拼写错误的句子列表，然后我们将循环遍历每个句子，要求模型校对并进行纠正。我们要使用一些分隔符。最后获取响应并将其打印出来。

In [ ]:
text = [ 
    "The girl with the black and white puppies have a ball.", # The girl has a ball.
    "Yolanda has her notebook.", # ok
    "Its going to be a long day. Does the car need it’s oil changed?", # Homonyms
    "Their goes my freedom. There going to bring they’re suitcases.", # Homonyms
    "Your going to need you’re notebook.", # Homonyms
    "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
    "This phrase is to cherck chatGPT for speling abilitty" # spelling
]

for t in text:
    prompt = f"Proofread and correct: ```{t}```"
    response = get_completion(prompt)
    print(response) 

运行程序，模型输出如下。

```
The girl with the black and white puppies has a ball.
Yolanda has her notebook.
"It's going to be a long day. Does the car need its oil changed?"
Here is the corrected version: "There goes my freedom. They are going to bring their suitcases."
You're going to need your notebook.
"That medicine affects my ability to sleep. Have you heard of the butterfly effect?"
This phrase is to check ChatGPT for spelling ability.
```

就这样，这个模型能够纠正所有这些语法错误。

我们可以使用一些我们在之前讨论过的技术来改进提示。为了改进提示，我们可以说，校对和纠正以下文本， 并重写整个校正后的版本。如果没有发现任何错误，只需输出“没有发现错误”。

In [ ]:
text = [ 
    "The girl with the black and white puppies have a ball.", # The girl has a ball.
    "Yolanda has her notebook.", # ok
    "Its going to be a long day. Does the car need it’s oil changed?", # Homonyms
    "Their goes my freedom. There going to bring they’re suitcases.", # Homonyms
    "Your going to need you’re notebook.", # Homonyms
    "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
    "This phrase is to cherck chatGPT for speling abilitty" # spelling
]

for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response) 

让我们来试试这个提示。通过这种方式，我们能够. . . 哦，这里还在使用引号。

```
The girl with the black and white puppies has a ball.
No errors found
It's going to be a long day. Does the car need its oil changed?
No errors found
You're going to need your notebook.
No errors found
This phrase is to check ChatGPT for spelling ability.
```

通过这种方式，我们能够. . . 哦，这里还在使用引号。

但你可以想象，通过一点点迭代地进行提示开发，你能够找到一个更加可靠的提示方式，每一次都能更好地工作。

现在我们再举一个例子。在你把文本发布到公共论坛之前，检查一下总是很有用的。因此，我们将举一个检查评论的例子。下面是一篇关于毛绒熊猫玩具的评论。我们将要求模型校对和纠正这篇评论。

In [ ]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room. Yes, adults also like pandas too. She takes \
it everywhere with her, and it's super soft and cute. One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price. It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""

prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)
 

很好。所以我们有了这个纠正的版本。

I got this for my daughter for her birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.

我们还可以做一个很酷的事情，就是找到原始评论和模型输出之间的差异。我们将使用 RedLines Python 包来实现这个功能。我们将获取评论的原始文本和模型输出之间的差异，然后显示出来。

In [ ]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown)) 

预期在下面可以看到原始评论和模型输出之间的差异，以及已经纠正的内容（红色）。我们在这里使用的提示是，校对并更正这篇评论。

![html](./img/6-2.jpg)

你也可以做一些更戏剧性的改变，例如语气的改变等等。让我们再尝试一下。

在这个提示中，我们要求模型校对和更正这篇相同的评论，但也要求对内容进行修改使其更有说服力，并确保它遵循 APA 风格。针对高级读者。我们还将要求以 markdown 格式输出。在这里我们使用与原始评论相同的文本。

In [ ]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""

response = get_completion(prompt)
display(Markdown(response)) 

我们来执行这个操作。

预期有一个扩展的 APA 样式的评论，关于毛绒熊猫。

![review](./img/6-3.jpg)

这就是关于文本转换任务的全部内容。接下来，我们将进行扩写任务，我们将使用较短的提示，从语言模型中生成更长、更自由的响应。

